In [159]:
import pandas as pd
import numpy as np
import utils_local

from scipy import stats

from surprise import Dataset, Reader
from surprise import KNNBasic, KNNWithMeans
from surprise.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.metrics import root_mean_squared_error, mean_squared_error, mean_absolute_error, f1_score

In [160]:
# Genera and sites
path = "../data/AllSites_SiteOccurrences_AllGenera_26.1.24.csv"

df = pd.read_csv(path)
df.head()

,SITE_NAME,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,Aba Zawei,1,1,1,1,0,0,0,0,0,...,4,4,0,0.0,0.000000,4,0,NaN,1.000000,0.0265
1,Abric Romani,1,0,1,0,1,1,1,1,1,...,12,12,0,0.0,0.000000,6,5,1.2,0.500000,0.0550
2,Acheng_Jiaojie,0,0,0,0,0,0,1,0,0,...,7,5,2,0.4,0.285714,5,2,2.5,0.714286,0.2100
3,Adler cave,1,0,0,0,0,1,0,1,0,...,10,5,5,1.0,0.500000,6,4,1.5,0.600000,0.0275
4,Adyrgan,1,0,0,1,0,0,0,0,0,...,11,5,6,1.2,0.545455,11,0,NaN,1.000000,2.2000


In [161]:
# Dental traits genera
path = "../data/DentalTraits_Genus_PPPA.csv"

df_dental = pd.read_csv(path, sep=",")
df_dental.head()

,Genus,n,Mass.g_Mean,Diet.Plant_Mean,HYP_Mean,LOP_Mean,FCT_AL_Mean,FCT_OL_Mean,FCT_SF_Mean,BUN_Mean,...,HYP_Mode,LOP_Mode,FCT_AL_Mode,FCT_OL_Mode,FCT_SF_Mode,BUN_Mode,Order,ConsInGenhyp,ConsInGen_bun,Unnamed: 21
0,Addax,1.0,70000.300000,100.0,3.0,2.0,0.0,1.0,1.0,0.0,...,3,2,0,1,1.0,0,Cetartiodactyla,True,True,NaN
1,Aepyceros,1.0,52500.100000,100.0,3.0,2.0,0.0,1.0,0.0,0.0,...,3,2,0,1,0.0,0,Cetartiodactyla,True,True,NaN
2,Alcelaphus,1.0,171001.500000,100.0,3.0,2.0,0.0,1.0,0.0,0.0,...,3,2,0,1,0.0,0,Cetartiodactyla,True,True,NaN
3,Alces,1.0,356998.000000,100.0,1.0,2.0,1.0,1.0,0.0,0.0,...,1,2,1,1,0.0,0,Cetartiodactyla,True,True,NaN
4,Allochrocebus,3.0,5708.333333,90.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0.0,1,Primates,True,True,NaN


In [162]:
# Mass-diet
path = "../data/FossilGenera_MammalMassDiet_Jan24.csv"

df_mass_diet = pd.read_csv(path, sep=",")
df_mass_diet.head()

,Genus,Order,Family,MassSource,Massg,LogMass,LargeSmall,SizeClass,Diet,DietSource
0,Abudhabia,Rodentia,Muridae,Family average,1.343147e+02,2.128124,Small,small,Herbivore,Phylacine
1,Aceratherium,Perissodactyla,Rhinocerotidae,Cooke,1.099006e+06,6.041000,Large,large,Herbivore,Phylacine
2,Acinonyx,Carnivora,Felidae,Phylacine,4.670000e+04,4.669317,Large,large,Non-Herbivore,Phylacine
3,Aepyosciurus,Rodentia,Sciuridae,NOW,2.860000e+02,2.456366,Small,small,Herbivore,Phylacine
4,Aeretes,Rodentia,Sciuridae,Phylacine,7.324000e+02,2.864748,Small,small,Herbivore,Phylacine


Evaluation similarly as in paper: Collaborative Filtering for Implicit Feedback Datasets ?

## Evalaution

In [163]:
def f(df_species: pd.DataFrame):
    if np.sum(df_species["occurence"]) == 0:
        return 0

    preferences = df_species["pred"]
    percentile_rank = 1 - stats.percentileofscore(preferences, preferences) / 100

    expected_rank_species = np.sum(df_species["occurence"] * percentile_rank) / np.sum(df_species["occurence"])

    return expected_rank_species

In [164]:
def calc_expected_percentile_rank(df_pred: pd.DataFrame) -> float:
    """Calculate the expected percentile rank as in paper "Collaborative Filtering for Implicit Feedback Datasets"

    Args:
        df_pred (pd.DataFrame): prediction dataframe

    Returns:
        float: expected percentile rank
    """
    expected_ranks = (
        df_pred.sort_values(by="pred")
        .groupby(by="genus")
        .apply(f, include_groups=False)
    )
    expected_percentile_rank = expected_ranks[expected_ranks > 0].mean()

    return expected_percentile_rank

## Building content-based filtering

Add an option to recommend n (5) more genera on top of the "normal" cutoff and threshold

In [165]:
def build_genus_info_from_genus_data(df):
    genus_info_cols = [
        "Genus",
        "Order",
        "Family",
        "Massg",
        "Diet",
       # "DietSource"
    ]
    
    df_genus_info = df[genus_info_cols]

    dummy_cols = [
        "Order",
        "Family",
        "Diet",
       # "DietSource"
    ]

    df_genus_info = pd.get_dummies(df_genus_info, columns=dummy_cols)
    df_genus_info = df_genus_info.replace({False: 0, True: 1})
    df_genus_info = df_genus_info.rename(columns={"Genus": "genus"})

    return df_genus_info

In [166]:
def build_site_genus_matrix(df):
    cols_redundant = ['LAT',
    'LONG',
    'ALTITUDE',
    'MAX_AGE',
    'BFA_MAX',
    'BFA_MAX_ABS',
    'MIN_AGE',
    'BFA_MIN',
    'BFA_MIN_ABS',
    'COUNTRY',
    'age_range',
    'Total_Gen_Count',
    'Large_GenCount',
    'Small_GenCount',
    'smallperlarge',
    'smallprop',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'DietRatio',
    'HerbProp',
    'mid_age'
    ]


    df_site_genus = df.drop(columns=cols_redundant).set_index('SITE_NAME')

    return df_site_genus

In [167]:
def build_site_info(df):
    site_info_cols = [
    'SITE_NAME',
    'LAT',
    'LONG',
    'MAX_AGE',
    'MIN_AGE',
    'age_range',
    'Large_GenCount',
    'Small_GenCount',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'mid_age'
    ]

    df_site_info = df[site_info_cols].set_index('SITE_NAME')
    
    return df_site_info

In [168]:
def build_genus_info(df, df_genus_data):
    site_genus = build_site_genus_matrix(df)
    site_genus = site_genus.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})
    site_info = build_site_info(df)

    genus_info = site_genus.merge(site_info, on="SITE_NAME", how="left")
    genus_info = genus_info[genus_info["presence"] == 1]

    genus_info = genus_info.drop(["SITE_NAME", "presence"], axis=1)
    genus_info = genus_info.groupby('genus').mean().reset_index().set_index("genus")

    df_genus_data = build_genus_info_from_genus_data(df_genus_data)
    genus_info = genus_info.merge(df_genus_data, left_index=True, right_on="genus", how="left").reset_index(drop=True).set_index("genus")

    return genus_info

In [169]:
# Building infro about the genera for each site (in progress)
def build_site_info_from_genus_data(df, df_mass_diet):
    genus_info = build_genus_info_from_genus_data(df_mass_diet)
    site_genus = build_site_genus_matrix(df)

    site_genus = site_genus.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})
    site_genus = site_genus[site_genus["presence"] == 1].drop("presence", axis="columns")

    site_genus = site_genus.merge(genus_info, on="genus", how="left")
    site_genus = site_genus.drop(["genus"], axis=1)
    site_genus = site_genus.groupby('SITE_NAME').mean().reset_index().set_index("SITE_NAME")
    
    return site_genus


In [170]:
def build_site_info_with_genus_info(df, df_genus_data):
    site_info_cols = [
    'SITE_NAME',
    'LAT',
    'LONG',
    'MAX_AGE',
    'MIN_AGE',
    'age_range',
    'Large_GenCount',
    'Small_GenCount',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'mid_age'
    ]

    df_site_info = df[site_info_cols].set_index('SITE_NAME')
    
    df_site_info_by_genera = build_site_info_from_genus_data(df, df_genus_data)
    df_site_info = df_site_info.merge(df_site_info_by_genera, left_index=True, right_on="SITE_NAME", how="left")
    
    return df_site_info

In [171]:
def get_recommendations_for_site(genus_info, site_name, site_indices, genus_site_similarity_matrix, num_recommend = 10):
    idx = site_indices[site_name]

    # Sorted similarity scores
    sim_scores = sorted(list(enumerate(genus_site_similarity_matrix[:,idx])), key=lambda x: x[1], reverse=True)

    # Get the scores of the num_recommend most similar sites
    similar_genus_for_site = sim_scores[:num_recommend]

    # Get the genus indices
    genus_indices = [i[0] for i in similar_genus_for_site]
    genus_site_similarities = [i[1] for i in similar_genus_for_site]

    recommended_genus = genus_info.iloc[genus_indices].index.to_frame(index=False).assign(similarity=genus_site_similarities)
    recommended_genus.insert(0, "SITE_NAME", site_name)

    return recommended_genus

In [172]:
def normalize_columns_min_max(df):
    return (df - df.min()) / (df.max() - df.min())

In [173]:
def normalize_columns_mean(df):
    return (df - df.mean()) / df.std()

In [174]:
def find_recommendations_for_all_sites(df, df_genus_data, n_species_to_recommend, normalization: None):
    genus_info = build_genus_info(df, df_genus_data)
    site_info = build_site_info_with_genus_info(df, df_genus_data)
    
    if normalization != None:
        genus_info = normalization(genus_info)
        site_info = normalization(site_info)
    
    if genus_info.isnull().values.any():
        print("WARNING! Genus info data contains nans. Assigning to zeros")
        genus_info = genus_info.fillna(0)
    
    if site_info.isnull().values.any():
        print("WARNING! Site info data contains nans. Assigning to zeros")
        site_info = site_info.fillna(0)


    site_indices = pd.Series(df.index, index=df["SITE_NAME"]).drop_duplicates()
    sim = cosine_similarity(genus_info, site_info)

    recommendations = []
    for site, idx in site_indices.items():
        site_recommendations = get_recommendations_for_site(
            genus_info=genus_info,
            site_name=site,
            site_indices=site_indices,
            genus_site_similarity_matrix=sim,
            num_recommend=n_species_to_recommend                        
        )

        recommendations.append(site_recommendations)
    
    return pd.concat(recommendations).reset_index(drop=True)

In [175]:
all_site_recommendations = find_recommendations_for_all_sites(df, df_mass_diet, 414, normalization=normalize_columns_min_max)
all_site_recommendations_matrix = pd.pivot(all_site_recommendations, index="SITE_NAME", columns="genus", values="similarity").fillna(0)
all_site_recommendations_matrix.head()

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/4086351798.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_genus_info = df_genus_info.replace({False: 0, True: 1})
/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/4086351798.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_genus_info = df_genus_info.replace({False: 0, True: 1})


WARNING! Genus info data contains nans. Assigning to zeros
WARNING! Site info data contains nans. Assigning to zeros


genus,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,Alilepus,...,Villanyia,Viverra,Viverravus,Viverricula,Vormela,Vulpes,Wushanomys,Xenocyon,Yangia,Zygolophodon
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.180043,0.364299,0.445845,0.219695,0.212262,0.390973,0.550427,0.553821,0.000000,0.389376,...,0.379635,0.214788,0.213867,0.216596,0.168772,0.173730,0.378530,0.164339,0.223829,0.407516
Abric Romani,0.566466,0.356161,0.379461,0.572111,0.487954,0.354519,0.456357,0.582281,0.416029,0.306577,...,0.326927,0.482849,0.482316,0.488196,0.499435,0.655447,0.352848,0.605723,0.000000,0.297664
Acheng_Jiaojie,0.422511,0.453466,0.516303,0.470611,0.456165,0.461507,0.400687,0.617983,0.475462,0.663275,...,0.463813,0.462437,0.453840,0.459359,0.556993,0.423214,0.469126,0.405154,0.369447,0.447486
Adler cave,0.397478,0.378853,0.451871,0.389543,0.386860,0.452960,0.358536,0.496963,0.351992,0.591665,...,0.401224,0.393380,0.399579,0.385794,0.408016,0.578033,0.405792,0.525692,0.000000,0.309860
Adyrgan,0.466275,0.778500,0.558725,0.361456,0.000000,0.732205,0.638578,0.479227,0.333810,0.709398,...,0.883731,0.331066,0.000000,0.318378,0.409690,0.331502,0.816011,0.455917,0.769099,0.726559


### Performance

In [176]:
site_genus_matrix = build_site_genus_matrix(df)
df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

all_site_recommendations_and_real = all_site_recommendations.merge(
    df_site_genus,
    how="outer",
    on=["SITE_NAME", "genus"]
).fillna(0)

all_site_recommendations_and_real = all_site_recommendations_and_real.rename(
    columns={"similarity": "pred", "presence": "occurence"}
)

all_site_recommendations_and_real_sorted = all_site_recommendations_and_real.sort_values(by=["SITE_NAME", "pred"], ascending=[True, False]).head(10)
all_site_recommendations_and_real_sorted.head(10)

,SITE_NAME,genus,pred,occurence
82,Aba Zawei,Coelodonta,0.808066,1
129,Aba Zawei,Equus,0.794929,1
364,Aba Zawei,Rhinoceros,0.774805,0
99,Aba Zawei,Dicerorhinus,0.753962,0
336,Aba Zawei,Procapra,0.735801,0
405,Aba Zawei,Stephanorhinus,0.733861,0
372,Aba Zawei,Saiga,0.725962,0
282,Aba Zawei,Ovis,0.720584,0
281,Aba Zawei,Ovibos,0.716926,0
350,Aba Zawei,Pseudois,0.715576,0


In [177]:
def recommend_n(recommendation_and_real: pd.DataFrame, n: int=10):
    def top_n(df, n, column='pred'):
        return df.nlargest(n, column)
    
    recommendation_and_real = recommendation_and_real.groupby(
        'SITE_NAME', group_keys=False
    ).apply(
        top_n, n
    ).sort_values(
        by=["SITE_NAME", "pred"], ascending=[True, False]
    )

    return recommendation_and_real

recommend_n(all_site_recommendations_and_real, 10)

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/3122930816.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(


,SITE_NAME,genus,pred,occurence
82,Aba Zawei,Coelodonta,0.808066,1
129,Aba Zawei,Equus,0.794929,1
364,Aba Zawei,Rhinoceros,0.774805,0
99,Aba Zawei,Dicerorhinus,0.753962,0
336,Aba Zawei,Procapra,0.735801,0
...,...,...,...,...
324196,zhoukoudian_Loc18,Eirictis,0.805448,0
324263,zhoukoudian_Loc18,Hypolagus,0.800639,1
324475,zhoukoudian_Loc18,Sivapanthera,0.788398,0
324399,zhoukoudian_Loc18,Pliolagus,0.788361,0


In [178]:
def recommend_using_cutoff(recommendation_and_real: pd.DataFrame, cutoff: float=0.75):
    recommendation_and_real = recommendation_and_real.groupby(
        'SITE_NAME', group_keys=False
    ).apply(
        lambda x: x[x['pred'] >= cutoff]
    ).sort_values(
        by=["SITE_NAME", "pred"], ascending=[True, False]
    )
    
    return recommendation_and_real

recommend_using_cutoff(all_site_recommendations_and_real, 0.75)

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/1226959580.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(


,SITE_NAME,genus,pred,occurence
82,Aba Zawei,Coelodonta,0.808066,1
129,Aba Zawei,Equus,0.794929,1
364,Aba Zawei,Rhinoceros,0.774805,0
99,Aba Zawei,Dicerorhinus,0.753962,0
2150,Adyrgan,Promimomys,0.891514,1
...,...,...,...,...
324533,zhoukoudian_Loc18,Xenocyon,0.760099,1
324358,zhoukoudian_Loc18,Nyctereutes,0.759231,0
324085,zhoukoudian_Loc18,Aepyosciurus,0.756777,0
324157,zhoukoudian_Loc18,Chardinomys,0.753189,0


In [179]:
def recommend_n_more(recommendations_and_real: pd.DataFrame, n: int):
    def filter_rows(group):
        sorted_group = group.sort_values(by='pred', ascending=False)
        occurence_1 = sorted_group[sorted_group['occurence'] == 1]
        occurence_0_top_n = sorted_group[sorted_group['occurence'] == 0].head(n)
        return pd.concat([occurence_1, occurence_0_top_n])
    
    recommendations_and_real = recommendations_and_real.groupby('SITE_NAME').apply(filter_rows)
    recommendations_and_real.reset_index(drop=True, inplace=True)
    
    return recommendations_and_real

test_df = recommend_n_more(all_site_recommendations_and_real, 5)
test_df.head(50)

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/83520888.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  recommendations_and_real = recommendations_and_real.groupby('SITE_NAME').apply(filter_rows)


,SITE_NAME,genus,pred,occurence
0,Aba Zawei,Coelodonta,0.808066,1
1,Aba Zawei,Equus,0.794929,1
2,Aba Zawei,Bos,0.712633,1
3,Aba Zawei,Gazella,0.681797,1
4,Aba Zawei,Rhinoceros,0.774805,0
5,Aba Zawei,Dicerorhinus,0.753962,0
6,Aba Zawei,Procapra,0.735801,0
7,Aba Zawei,Stephanorhinus,0.733861,0
8,Aba Zawei,Saiga,0.725962,0
9,Abric Romani,Vulpes,0.655447,1


In [196]:
# Train-test split
PATH_DIR_DATA_PROCESS = "../data/data_processed/"

data_train = np.load(PATH_DIR_DATA_PROCESS + "data_train.npy", allow_pickle=True)
data_val = np.load(PATH_DIR_DATA_PROCESS + "data_val.npy", allow_pickle=True)

df_train = utils_local.conv_dataset_patch2df(data_train)
df_val = utils_local.conv_dataset_patch2df(data_val)

print(f"train: {df_train.shape}")
print(f"val: {df_val.shape}")

df_val.head()

train: (304000, 3)
val: (20536, 3)


,site,species,occurence
0,198,116,0.0
1,198,117,0.0
2,198,118,0.0
3,198,119,0.0
4,199,116,0.0


In [197]:
path_dir_encode = PATH_DIR_DATA_PROCESS

path_enc_genera = path_dir_encode + "ordinal_enc_species.json"
path_enc_site = path_dir_encode + "ordinal_enc_site.json"

enc_genera = utils_local.CategoryDict.from_file(path_enc_genera)
enc_site = utils_local.CategoryDict.from_file(path_enc_site)

In [198]:
enc_site.dict_id2name

{0: 'Aba Zawei',
 1: 'Abric Romani',
 2: 'Acheng_Jiaojie',
 3: 'Adler cave',
 4: 'Adyrgan',
 5: 'Akali',
 6: 'Akhalkalaki',
 7: 'Akhshtyrskaja cave: l. 2',
 8: 'Akhstyrskaya Cave',
 9: 'Alekseevsk',
 10: 'Almenara-Casablanca 1',
 11: 'Altestovo 1',
 12: 'An Nafud 2',
 13: 'Anetovka II',
 14: 'Angangxi_Daxingtun',
 15: 'Anping',
 16: 'Antu',
 17: 'Apollonia (APL)',
 18: 'Arenero de Casa de Eulogio',
 19: 'Arenero Manuel Soto',
 20: 'Arenero Oxigeno',
 21: 'Arondelli',
 22: 'Arta II 1-3',
 23: 'Arta II l.5',
 24: 'Asva',
 25: 'Atapuerca TD10',
 26: 'Atapuerca TD11',
 27: 'Atapuerca TD4',
 28: 'Atapuerca TD4-5',
 29: 'Atapuerca TD5',
 30: 'Atapuerca TD6',
 31: 'Atapuerca TD8',
 32: 'Atapuerca TD8-9',
 33: 'Atapuerca TE-10',
 34: 'Atapuerca TE-11',
 35: 'Atapuerca TE-12',
 36: 'Atapuerca TE-13',
 37: 'Atapuerca TE-14',
 38: 'Atapuerca TE-19',
 39: 'Atapuerca TE-9',
 40: 'Atapuerca TG',
 41: 'Atapuerca TN',
 42: 'Atapuerca TZ',
 43: 'Aukshtadvaris',
 44: 'Aveley',
 45: 'Azykh Cave: Unit I-V

In [199]:
df_train["site"] = df_train["site"].map(enc_site.dict_id2name)
df_train["species"] = df_train["species"].map(enc_genera.dict_id2name)

df_val["site"] = df_val["site"].map(enc_site.dict_id2name)
df_val["species"] = df_val["species"].map(enc_genera.dict_id2name)

In [200]:
df_train = pd.pivot(df_train, index="site", columns="species", values="occurence").fillna(0)

In [206]:
cols_redundant = ["SITE_NAME",
    'LAT',
    'LONG',
    'ALTITUDE',
    'MAX_AGE',
    'BFA_MAX',
    'BFA_MAX_ABS',
    'MIN_AGE',
    'BFA_MIN',
    'BFA_MIN_ABS',
    'COUNTRY',
    'age_range',
    'Total_Gen_Count',
    'Large_GenCount',
    'Small_GenCount',
    'smallperlarge',
    'smallprop',
    'Herb_GenCount',
    'Nonherb_GenCount',
    'DietRatio',
    'HerbProp',
    'mid_age'
    ]

df_redundant = df[cols_redundant]
df_train = df_train.merge(df_redundant, how="left", left_index=True, right_on="SITE_NAME")
df_train

,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alcelaphus,Alces,Algarolutra,Alilepus,...,Total_Gen_Count,Large_GenCount,Small_GenCount,smallperlarge,smallprop,Herb_GenCount,Nonherb_GenCount,DietRatio,HerbProp,mid_age
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4,4,0,0.000000,0.000000,4,0,NaN,1.000000,0.02650
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12,12,0,0.000000,0.000000,6,5,1.200000,0.500000,0.05500
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7,5,2,0.400000,0.285714,5,2,2.500000,0.714286,0.21000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,5,5,1.000000,0.500000,6,4,1.500000,0.600000,0.02750
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11,5,6,1.200000,0.545455,11,0,NaN,1.000000,2.20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5,0,5,NaN,1.000000,0,5,0.000000,0.000000,1.55000
717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9,2,7,3.500000,0.777778,9,0,NaN,1.000000,2.31250
283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16,14,2,0.142857,0.125000,5,11,0.454545,0.312500,0.95585
299,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,24,24,0,0.000000,0.000000,13,10,1.300000,0.541667,0.95585


In [252]:
all_site_recommendations = find_recommendations_for_all_sites(df_train, df_mass_diet, 414, normalization=normalize_columns_min_max)
all_site_recommendations_matrix = pd.pivot(all_site_recommendations, index="SITE_NAME", columns="genus", values="similarity").fillna(0)
all_site_recommendations_matrix.head()

/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/4086351798.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_genus_info = df_genus_info.replace({False: 0, True: 1})
/var/folders/5m/7ljzc1r113358z1s1_rs2k4r0000gn/T/ipykernel_91094/4086351798.py:21: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_genus_info = df_genus_info.replace({False: 0, True: 1})


WARNING! Genus info data contains nans. Assigning to zeros
WARNING! Site info data contains nans. Assigning to zeros


genus,Acinonyx,Aepyosciurus,Aeretes,Ailuropoda,Ailurus,Alactagulus,Alces,Algarolutra,Alilepus,Allactaga,...,Villanyia,Viverra,Viverravus,Viverricula,Vormela,Vulpes,Wushanomys,Xenocyon,Yangia,Zygolophodon
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.194809,0.388483,0.475441,0.234022,0.226337,0.416928,0.589521,0.000000,0.402237,0.432884,...,0.404837,0.229014,0.228063,0.230965,0.180946,0.184132,0.403955,0.175097,0.238688,0.425480
Abric Romani,0.583633,0.381101,0.406031,0.573373,0.508807,0.379344,0.620126,0.432449,0.294627,0.388929,...,0.349820,0.503651,0.503389,0.509503,0.508263,0.659950,0.377434,0.611518,0.000000,0.313403
Acheng_Jiaojie,0.447402,0.486100,0.553458,0.495395,0.479939,0.494720,0.655905,0.501035,0.588636,0.509407,...,0.497192,0.486869,0.477867,0.483661,0.587397,0.442813,0.503189,0.425954,0.396034,0.474969
Adler cave,0.447782,0.436906,0.521112,0.442131,0.438215,0.522368,0.570309,0.398363,0.523062,0.530194,...,0.462705,0.445921,0.453245,0.437244,0.453387,0.624962,0.468738,0.569977,0.000000,0.356778
Adyrgan,0.484808,0.800589,0.574577,0.371582,0.000000,0.752981,0.492868,0.343269,0.734358,0.730980,...,0.908807,0.340433,0.000000,0.327404,0.426996,0.340941,0.837980,0.468729,0.790922,0.715151


In [259]:
all_site_recommendations_matrix.to_csv("content-based-filtering.csv")

In [253]:
site_genus_matrix = build_site_genus_matrix(df)
df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

all_site_recommendations_and_real = all_site_recommendations.merge(
    df_site_genus,
    how="outer",
    on=["SITE_NAME", "genus"]
).fillna(0)

all_site_recommendations_and_real = all_site_recommendations_and_real.rename(
    columns={"similarity": "pred", "presence": "occurence"}
)

all_site_recommendations_and_real_sorted = all_site_recommendations_and_real.sort_values(by=["SITE_NAME", "pred"], ascending=[True, False]).drop(columns=["occurence"])
all_site_recommendations_and_real_sorted.head(10)

,SITE_NAME,genus,pred
82,Aba Zawei,Coelodonta,0.831578
364,Aba Zawei,Rhinoceros,0.800825
336,Aba Zawei,Procapra,0.784554
99,Aba Zawei,Dicerorhinus,0.782175
372,Aba Zawei,Saiga,0.774016
282,Aba Zawei,Ovis,0.768084
129,Aba Zawei,Equus,0.765137
281,Aba Zawei,Ovibos,0.763277
350,Aba Zawei,Pseudois,0.762953
401,Aba Zawei,Spirocerus,0.760499


In [254]:
df_val = df_val.rename(columns={"site": "SITE_NAME", "species": "genus"})
df_val

,SITE_NAME,genus,occurence
0,Grays Thurrock,Elephas,0.0
1,Grays Thurrock,Hystrix,0.0
2,Grays Thurrock,Cuon,0.0
3,Grays Thurrock,Rhinolophus,0.0
4,Grosse Grotte (Blaubeuren),Elephas,0.0
...,...,...,...
20531,La Malouteyre (Le Puy),Mimomys,0.0
20532,La Parte,Sorex,0.0
20533,La Parte,Cricetulus,0.0
20534,La Parte,Sinotherium,0.0


In [260]:
df_validate = df_val.merge(all_site_recommendations_and_real_sorted, on=["SITE_NAME", "genus"], how="left").sort_values(by=["SITE_NAME", "pred"], ascending=[True, False])
# df_validate.loc[df_validate['pred'] >= 0.6, 'pred'] = 1
# df_validate.loc[df_validate['pred'] < 0.6, 'pred'] = 0

In [261]:
calc_expected_percentile_rank(df_validate)

0.36855893007727875

In [251]:
df_validate

,SITE_NAME,genus,occurence,pred
19801,Aba Zawei,Procapra,0.0,1.0
15883,Aba Zawei,Bison,0.0,1.0
6473,Aba Zawei,Ammotragus,0.0,1.0
15403,Aba Zawei,Hippopotamus,0.0,0.0
11192,Aba Zawei,Asoletragus,0.0,0.0
...,...,...,...,...
18002,zhoukoudian_Loc18,Pygathrix,0.0,0.0
937,zhoukoudian_Loc18,Pliomys,0.0,0.0
4152,zhoukoudian_Loc18,Muva,0.0,0.0
18000,zhoukoudian_Loc18,Szechuanopithecus,0.0,0.0


#### Expected percentile rank

In [62]:
calc_expected_percentile_rank(all_site_recommendations_and_real)

0.08383868307757367

In [27]:
# Find missing columns from recommendations

df_site_genus = build_site_genus_matrix(df)
missing_columns = df_site_genus.columns.difference(all_site_recommendations_matrix.columns)

for col in missing_columns:
    all_site_recommendations_matrix[col] = 0

In [28]:
column_order = df_site_genus.columns
all_site_recommendations_matrix = all_site_recommendations_matrix[column_order]
all_site_recommendations_matrix

genus,Equus,Coelodonta,Bos,Gazella,Ursus,Vulpes,Cervus,Canis,Sus,Homo,...,Euarctos,Paracervulus,Eostyloceros,Cervocerus,Antispiroides,Sinoryx,Prospalax,Pliopetaurista,Predicrostonyx,Boocercus
SITE_NAME,,,,,,,,,,,,,,,,,,,,,
Aba Zawei,0.794929,0.808066,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
Abric Romani,0.000000,0.000000,0.0,0.0,0.0,0.655447,0.0,0.644547,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
Acheng_Jiaojie,0.000000,0.642129,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
Adler cave,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
Adyrgan,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zuurland (-42 to -46 m),0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0
Zverinogolovskoe,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.833241,0.0,0
kudaro 3 cave l.3,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0


In [29]:
all_site_recommendations_matrix[all_site_recommendations_matrix > 0.5] = 1


In [31]:
rms = root_mean_squared_error(df_site_genus, all_site_recommendations_matrix)
mae = mean_absolute_error(df_site_genus, all_site_recommendations_matrix)
# f1 = f1_score(df_site_genus, all_site_recommendations_matrix, zero_division=1.0, average='macro')
print(rms, mae)#, f1)

0.17743505886287805 0.04533368242018128


## Building collaborative filtering

In [ ]:
def fit_knn_collaborative_filtering(df, site_based=True, similarity="cosine", k=5, min_k=1):
    site_genus_matrix = build_site_genus_matrix(df)
    df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(df_site_genus, reader) # Column order must be user, item, rating

    sim_options = {
    'name': similarity,
    'user_based': site_based  # True for user-user, False for item-item
    }

    trainset = data.build_full_trainset()

    knn = KNNBasic(k=k, min_k=min_k, sim_options=sim_options)
    knn.fit(trainset)

    return knn, trainset

In [ ]:
knn, trainset = fit_knn_collaborative_filtering(
    df, 
    site_based=True, 
    similarity="cosine",
    k=5,
    min_k=1
)

In [ ]:
def predict_collaborative_filtering(algotihm, trainset):
    # Testset equal to trainset
    testset = trainset.build_testset()

    # Get predictions for all user-item pairs
    predictions = algotihm.test(testset)

    # Get item scores from the predictions
    item_scores = [(prediction.uid, prediction.iid, prediction.est) for prediction in predictions]
    return pd.DataFrame(item_scores, columns =['SITE_NAME', 'PREDICTED_GENUS', 'SCORE'])

In [ ]:
predictions = predict_collaborative_filtering(knn, trainset)

In [ ]:
display(predictions.head(10))

In [ ]:
items_scores_matrix_knn = pd.pivot(predictions, index="SITE_NAME", columns="PREDICTED_GENUS", values="SCORE")
items_scores_matrix_knn.head()

### Performance

In [ ]:
predictions = predictions.rename(
    columns={"PREDICTED_GENUS": "genus"}
)

site_genus_matrix = build_site_genus_matrix(df)
df_site_genus = site_genus_matrix.stack().reset_index().rename(columns={"level_1": "genus", 0: "presence"})

all_site_recommendations_and_real = predictions.merge(
    df_site_genus,
    how="outer",
    on=["SITE_NAME", "genus"]
).fillna(0)

all_site_recommendations_and_real = all_site_recommendations_and_real.rename(
    columns={"SCORE": "pred", "presence": "occurence"}
)

all_site_recommendations_and_real.head()

In [ ]:
calc_expected_percentile_rank(all_site_recommendations_and_real)

In [ ]:
column_order = df_site_genus.columns
items_scores_matrix_knn = items_scores_matrix_knn[column_order]
items_scores_matrix_knn

In [ ]:
items_scores_matrix_knn[items_scores_matrix_knn > 0.5] = 1
items_scores_matrix_knn[items_scores_matrix_knn <= 0.5] = 0

In [ ]:
items_scores_matrix_knn

In [ ]:
rms = root_mean_squared_error(df_site_genus, items_scores_matrix_knn)
mae = mean_absolute_error(df_site_genus, items_scores_matrix_knn)
f1 = f1_score(df_site_genus, items_scores_matrix_knn, zero_division=1.0, average='macro')
print(rms, mae, f1)

## Hybrid algorithm content-knn